## How many people has ICE arrested in North Carolina?

The answer to this question is not simple. Nearly 20% of the rows in the ICE arrests database are missing values for State of Apprehension. So one cannot simply filter for North Carolina and call it a day. For the rows missing state values, you need to use a combination of the Area of Responsibility and the Apprehension Site Landmark to find everyone.

This colab notebook goes through each step to catch as many North Carolina arrests as possible.

The original files can be found here. https://drive.google.com/drive/folders/1ANA4ibvcbvohOfuXWv4uqaUDaLmQwIoz?usp=sharing

The source is the Deportation Data Project. https://deportationdata.org/data.html 





In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
np.set_printoptions(threshold=np.inf)

In [2]:
df = pd.read_excel('/Users/cmcglade/Documents/Projects/north-carolina-ice-arrests/og_data/ice_release_11aug2025/2025-ICLI-00019_2024-ICFO-39357_ERO Admin Arrests_LESA-STU_FINAL Redacted_raw.xlsx', skiprows=6)

In [6]:
#create a column that flags Jan 20 through June in both 2024 and 2025

df['comparetime'] = np.where(
    df['Apprehension Date'].between('2025-01-20', '2025-06-30'),
    'this_year',
    np.where(
        df['Apprehension Date'].between('2024-01-20', '2024-06-30'),
        'last_year',
        'not in compare range'
    )
)

df = df[(df['comparetime'] == 'this_year') | (df['comparetime'] == 'last_year')]
noloc = df[
    df['Apprehension AOR'].isnull() &
    df['Apprehension State'].isnull() &
    df['Apprehension Site Landmark'].isnull()
]

print('Rows with missing location during my compare times:', noloc.shape)

Rows with missing location during my compare times: (1831, 24)


In [12]:
## remove possible duplicates

df = df.sort_values(['Unique Identifier', 'Apprehension Date'])

# Only apply to rows with valid IDs
has_id = df[df['Unique Identifier'].notna()].copy()
no_id = df[df['Unique Identifier'].isna()].copy()

# Calculate time differences per person
has_id['time_diff'] = has_id.groupby('Unique Identifier')['Apprehension Date'].diff()

# Flag rows that are within 24 hours of the previous record
has_id['within_24h'] = has_id['time_diff'] <= pd.Timedelta(hours=24)

# Keep the first record, and discard later ones within 24h of the same person
# (i.e., keep where within_24h is False or NaN)
df_kept = has_id[(~has_id['within_24h']) | (has_id['within_24h'].isna())]
df_discarded = has_id[has_id['within_24h']]


# Add back rows with missing IDs (we don't filter those)
df = pd.concat([df_kept, no_id], ignore_index=True)

df['approx_age'] = 2025-df['Birth Year']

#create a year column
df['year'] = df['Apprehension Date'].dt.year
df['year'] = df['year'].astype(str)

#create a month column
df['month'] = df['Apprehension Date'].dt.month
df['month'] = df['month'].astype(str)

#create a generic day column so I can group by month-year
df['day'] = '01'

#create a standardized date for month-year
df['standardized_date'] = df['year']+'-'+df['month']+'-'+df['day']
df['standardized_date'] = pd.to_datetime(df['standardized_date'])


In [11]:
#create a method category 
df['method_cat'] = np.where(
    df['Apprehension Method'].isin(['Non-Custodial Arrest', 'Located', 'Worksite Enforcement', 'Traffic Check', 'Crewman/Stowaway']),
    'non-custodial, located, traffic check or worksite enforcement',
    np.where(
        df['Apprehension Method'] == 'Probation and Parole',
        'probation and parole',
        np.where(
            df['Apprehension Method'].isin(['CAP Federal Incarceration', 'CAP State Incarceration', 'CAP Local Incarceration', 'Custodial Arrest']),
            'federal, state or local incarceration',
            np.where(
                df['Apprehension Method'].isin([
                    'Other efforts', 'Other Task Force', 'Other Agency (turned over to INS)',
                    'Patrol Interior', 'Law Enforcement Agency Response Unit'
                ]),
                'other',
                np.where(
                    df['Apprehension Method'] == 'ERO Reprocessed Arrest',
                    're-entry arrest',
                    np.where(
                        df['Apprehension Method'] == '287(g) Program',
                        '287g',
                        'other2'
                    )
                )
            )
        )
    )
)

## Step One: Break the file into two pieces

One file is missing state values and the other has them.

Filter for NC and set aside the the clean file for later, move the messy file on to the next step.

In [13]:
# Lowercase columns first
df['Apprehension Site Landmark'] = df['Apprehension Site Landmark'].str.lower()
df['Apprehension State'] = df['Apprehension State'].str.lower()
df['Apprehension AOR'] = df['Apprehension AOR'].str.lower()

#create two separate files, one where there is no state and the other where there is. I will clean the no_state file.
no_state = df[df['Apprehension State'].isnull()].copy()
print('No State: ', no_state.shape)


state = df[df['Apprehension State'].notna()].copy()
print('State: ', state.shape)

#filter for NC
nc1 = state[state['Apprehension State']== 'north carolina']

No State:  (28810, 32)
State:  (133892, 32)


## Step Two: Peel states out of the site landmark column

The site landmark column has a variety of location types. Sometimes, the state is referenced with a comma and the two letter abbreviation. Here, I specify valid state abbreviations and run a regex argument to pick up any of those and dump them into a new column called 'extracted state.'

I then create a new file that grabs all rows where this was successful, filtered for nc and set that aside. I also create a file where there was not a state abbreviation within the landmark column, so the extracted state column is blank.

That file, which I called no_state2 moves on to the next step.

In [14]:
# List of valid US state abbreviations
us_states = [
    'al','ak','az','ar','ca','co','ct','de','fl','ga','hi','id','il','in','ia','ks','ky','la',
    'me','md','ma','mi','mn','ms','mo','mt','ne','nv','nh','nj','nm','ny','nc','nd','oh','ok',
    'or','pa','ri','sc','sd','tn','tx','ut','vt','va','wa','wv','wi','wy'
]

# Extract potential 2-letter state code from "Apprehension Site Landmark"
no_state['extracted_state'] = (
    no_state['Apprehension Site Landmark']
      .str.extract(r',\s*([a-zA-Z]{2})(?![a-zA-Z])', expand=False)  # match exactly two letters after a comma
      .str.lower()                                                  # make them lowercase
      .where(lambda x: x.isin(us_states))                           # only keep valid US abbreviations
)
#create a dataframe that has state names now that didn't before
extractedstate = no_state[no_state['extracted_state'].notna()].copy()
print('Rows with extracted states from Site Landmark: ', extractedstate.shape)

#create a dataframe that is still missing a state
no_state2 = no_state[no_state['extracted_state'].isnull()].copy()
print('Rows where there is no Apprehension State and no clear state abbreviation in landmark: ', no_state2.shape)

extractedstate['Apprehension State'] = extractedstate['extracted_state']
extractedstate = extractedstate.drop(columns=['extracted_state'])
#extractedstate['Apprehension State'].unique()

#filter for nc
nc2 = extractedstate[extractedstate['Apprehension State'] == 'nc']

Rows with extracted states from Site Landmark:  (3613, 33)
Rows where there is no Apprehension State and no clear state abbreviation in landmark:  (25197, 33)


## Step Three: Search for North Carolina counties

Using the file where there was no state abbreviation found, and no state referenced in the Apprehension State column, I create a new file that grabs any rows where the Area of Responsibility is Atlanta.

[In a separate file](https://colab.research.google.com/drive/1644oPILC8rSwN2LFSWIYkOxGI0vIG85B?usp=sharing), I used the tigerline census shapefile to pull all the county names in North Carolina, Georgia and South Carolina (our AOR) and then ran a groupby to identify counties in North Carolina that share a name with counties in the other two states. I excluded those, and created a list of counties that only exist in North Carolina.

From here, I created a new file called nc_county_names that grabs any row where the counties in my list are mentioned. I set that aside for later.

I then also created a new file where those counties were not mentioned, and printed out the unique landmark variables and David Raynor looked at them and picked out several that were definitly in North Carolina, that I would not know about because I'm new here. I created a list of them, and then created a file that pulled those rows out called nc_stragglers.

I had also created a file where there was no Apprehension AOR listed. I looked at the unique variables there and found one location that was definitely in North Carolina. Raynor found some that could possibly be, but we couldn't be sure, so I excluded them.

The drawback of this is that I am likely missing people who were picked up in North Carolina, but simply because their county shares a name with another one in neighboring states, they are excluded. Or, in some places where the AOR is missing and the landmark doesn't provide obvious clues. There isn't really anything I can do about this, though, as there are not any other columns that would provide clues to confirm that the person was in North Carolina and not Georgia, or South Carolina.

At the very end of all of this, I stack my three dataframes into one: the file that found North Carolina county names, the file that found places that are in North Carolina and are in the AOR, and the file that found a few rows that were in North Carolina but were missing an AOR.





In [15]:
#create a dataframe that might contain NC arrests
potential_nc = no_state2[no_state2['Apprehension AOR'].str.contains('atlanta', na=False)]

no_aor = no_state2[no_state2['Apprehension AOR'].isnull()]

print('Rows where AOR is Atlanta', potential_nc.shape)
print('Rows where AOR is NULL', no_aor.shape)

nc_counties = [
'alamance', 'alexander', 'alleghany', 'anson', 'ashe', 'avery',
'bertie', 'bladen', 'brunswick', 'buncombe', 'cabarrus',
'caldwell', 'carteret', 'caswell', 'catawba', 'chowan', 'cleveland',
'columbus', 'craven', 'cumberland', 'currituck', 'dare', 'davidson',
'davie', 'duplin', 'durham', 'edgecombe', 'gaston', 'gates', 'graham',
'granville', 'guilford', 'halifax', 'harnett', 'haywood', 'henderson', 'hertford',
'hoke', 'hyde', 'iredell', 'johnston', 'lenoir', 'martin', 'mcdowell', 'mecklenburg',
'moore', 'nash', 'new hanover', 'northampton', 'onslow', 'orange', 'pamlico',
'pasquotank', 'pender', 'perquimans', 'person', 'pitt', 'robeson',
'rockingham', 'rowan', 'rutherford', 'sampson', 'scotland', 'stanly', 'stokes',
'surry', 'swain', 'transylvania', 'tyrrell', 'vance', 'wake',
'watauga', 'wilson', 'yadkin', 'yancey'

]

import re

nc_county_names = potential_nc[
    potential_nc['Apprehension Site Landmark']
    .str.lower()
    .str.contains('|'.join([re.escape(c) for c in nc_counties]), na=False)
]

import re

other_potential_nc = potential_nc[
    ~potential_nc['Apprehension Site Landmark']
    .str.lower()
    .str.contains('|'.join([re.escape(c) for c in nc_counties]), na=False)
]


print('Rows in NC counties and are also in the Atlanta AOR (dupe name counties excluded: )', nc_county_names.shape)
print('Rows that are in the Atlanta AOR but do not have any NC county names', other_potential_nc.shape)



defnc = [
    'butner bop facility',
    'nc department of corrections',
    'rdu general area, non-specific',
    'ero charlotte',
    'edgecomb county jail',
    'butner medical facility'
]

nc_stragglers = potential_nc[
    potential_nc['Apprehension Site Landmark'].str.lower().isin(defnc)
]

nc_stragglers2 = no_aor[no_aor['Apprehension Site Landmark'] == 'sc. nc line from mecklenburg co e to richmond co; n to nc/va line']

print('NC locations in Atlanta AOR that did not get picked up by county name: ', nc_stragglers.shape)
print('NC locations that had no AOR at all: ', nc_stragglers2.shape)
print('Rows where AOR is Atlanta but there is no landmark info: ', potential_nc[potential_nc['Apprehension Site Landmark'].isnull()].shape)

#no_aor['Apprehension Site Landmark'].unique()

nc3 = pd.concat([nc_stragglers, nc_stragglers2, nc_county_names])

Rows where AOR is Atlanta (368, 33)
Rows where AOR is NULL (1537, 33)
Rows in NC counties and are also in the Atlanta AOR (dupe name counties excluded: ) (39, 33)
Rows that are in the Atlanta AOR but do not have any NC county names (329, 33)
NC locations in Atlanta AOR that did not get picked up by county name:  (30, 33)
NC locations that had no AOR at all:  (16, 33)
Rows where AOR is Atlanta but there is no landmark info:  (4, 33)


## Step Four: Putting it all together

The first file I created (nc1) contained rows that all included the state name. The second file I created (nc2) has a state name as well, extracted from the landmark column.

I filtered for North Carolina in both of these.

The third file I created (nc3) combines rows where AOR is missing, but the landmark site had obvious ties to NC, AOR is Atlanta and the landmark had a NC-specific county name, AOR is Atlanta and the landmark had a place that was in North Carolina, identified by Raynor.

I stack these files on top of each other to make a master file called nc.

I have to do a little cleaning at this step, too.

There are duplicate rows, so I drop those. And then, I need to drop the rows where ICE clearly made a mistake when they listed the arrest in North Carolina, because they have the area of responsibility in other places and the arrest landmark also in other places.

The Deportation Data Project suggested people also dedupe rows where the apprehension date and the unique ID is repeated. I went a little further, noticing that some people have two arrests in one day, and decided to get rid of the second instance where someone was arrested twice in one day.

I merge the groupby back to the nc file and then run a command to keep only the first iteration of each unique ID.

I also discarded 36 rows where the same person was listed as arrested twice in the same month.

You end up with 4,595 arrests.

In [16]:
nc = pd.concat([nc1,nc2,nc3])
nc = nc.drop_duplicates()
nc = nc[
    (nc['Apprehension AOR'] == 'atlanta area of responsibility') |
    (nc['Apprehension AOR'].isnull())
]

nc.shape

(2677, 33)

In [25]:
#create a file for cleaning
landmarks = nc[['Apprehension Site Landmark']]
landmarks = landmarks.drop_duplicates()
landmarks.to_csv('/Users/cmcglade/Documents/Projects/north-carolina-ice-arrests/processed_data/landmarks.csv')

#clean the county names in Excel
#import the clean file back in
clean_landmarks = pd.read_csv('/Users/cmcglade/Documents/Projects/north-carolina-ice-arrests/processed_data/landmarks-clean.csv')
clean_landmarks.sample()

#join the cleaned names to the original file

nc = pd.merge(nc,clean_landmarks,how='left')

#create a column for in custody or not 

nc['in_custody_arrest'] = np.where(nc['method_cat']=='non-custodial, located, traffic check or worksite enforcement', 'n', 'y')




In [26]:
nc.to_csv('/Users/cmcglade/Documents/Projects/north-carolina-ice-arrests/processed_data/nc_ice_arrests.csv')